In [76]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.platypus import PageTemplate, PageBreak, SimpleDocTemplate


from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

import logging
import pandas as pd
from pathlib import Path

from pypdf import PdfMerger

doc = '6453'

In [77]:
def process_OPList(data):
    data['date_AC'] = data['date'].astype(int)
    data['date_AC'] = 19110000 + data['date_AC']
    data['date_AC'] = pd.to_datetime(data['date_AC'],
                                    format='%Y%m%d')
    data['Diagnosis'] = [st.replace('術前診斷:','') for st in data['Diagnosis']]
    data['Procedure'] = [st.replace('手術名稱:','') for st in data['Procedure']]
    data['IOL'] = [st.replace('手術室資訊:','') for st in data['OR info']]
    data['Anes'] = [st.replace('麻醉:','' ) for st in data['Anes']]
    return data.drop(columns=['change','OR info'])

In [78]:
logging.basicConfig(level=logging.INFO, filename='log.txt',
	format='[%(asctime)s %(levelname)-8s] %(message)s',
	datefmt='%Y%m%d %H:%M:%S',
	)

In [79]:
class Operation:
    def __init__(self,MRN,name,diagnosis,procedure,IOL):
        self.MRN = MRN
        self.name = name
        self.diagnosis = diagnosis.strip()
        self.procedure = procedure
        self.IOL = IOL
    def __str__(self):
        return f"MRN:{self.MRN},name{self.name}, dx:{self.diagnosis}, proc:{self.procedure}"

In [80]:
orders = [
    ["不剪睫毛"],
    ["Check Vital signs and Red eye"],
    ["Alcaine * 6"],
    ["Pheylephrine * 6"],
    ["Mydriacyl * 6"],
    ["Valium * 1"],
    ["Diamox * 1"],
    [f'開立醫師:{doc}'],
    ['執行護理師']
]

In [81]:
OP_list = pd.read_csv('OPList.csv')
OP_procecced = process_OPList(OP_list)
OP_today_df = OP_procecced.query("date_AC == '2023-05-01'")
OP_today_df
OP_today = []
for index, operation in OP_today_df.iterrows():
    OP_today.append(Operation(operation['MRN'],
                              operation['name'],
                              operation['Diagnosis'],
                              operation['Procedure'],
                              operation['IOL']))
for operation in OP_today:
    print(operation)

MRN:30282316,name張心亭, dx:CATA OD [全自費!!], proc: LENSX-PHACO-IOL OD, IVGA, [ICB ori+17.0] 
MRN:17377423,name洪仁道, dx:CATA OD, proc: PHACO-IOL OD [RAO600C Ori+10.5] 
MRN:48357842,name林張信子, dx:CATA OD, proc: PHACO-IOL OD [ICB Ori+20.0] 
MRN:24621849,name王明發, dx:CATA OD, proc: PHACO-IOL OD [AAB ori+10.0] 
MRN:29948376,name楊旭平, dx:CATA OS, proc: PHACO-IOL OS [ICB Ori+8.5] 
MRN:11911470,name關林阿女, dx:CATA OD, proc: PHACO-IOL OD 


In [83]:
width, height = A4
pdfmetrics.registerFont(TTFont('msj', 'msjhbd.ttc'))
new_doc_path = "preOPorderPDFs/result.pdf"
new_doc = SimpleDocTemplate(new_doc_path)
x_offset = 50
y_offset = 50
padding = 15


for i in range(len(OP_today)):
    new_page.setFont('msj',20)
    operation = OP_today[i]
    new_page = canvas.Canvas(new_doc_path,pagesize=A4)
    new_page.drawString(x_offset, height - y_offset, f"病歷號:{operation.MRN},姓名:{operation.name}")
    new_page.drawString(x_offset, height - y_offset * 2,
                       f"診斷:{operation.diagnosis}")
    new_page.drawString(x_offset, height - y_offset * 3,
                       f"術式:{operation.procedure}")
    new_page.drawString(
        x_offset, height - y_offset * 4,
        f'IOL:{operation.IOL}'
    )
    for k in range(len(orders)):
        new_page.drawString(x_offset, height - (k+5)*y_offset, orders[k][0])
    Story.append(PageBreak())        
new_doc.showPage()
new_doc.save()


NameError: name 'Story' is not defined

In [ ]:
pdf_folder = Path('preOPorderPDFs')
pdf_paths = list(pdf_folder.glob('*.pdf')) 

merger = PdfMerger()

for pdf_path in pdf_paths:
    with pdf_path.open() as pdf_file:
        merger.append(pdf_file)
merger.write('result.pdf')
merger.close()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 10: invalid start byte